In [30]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE, ADASYN

In [31]:
data = pd.read_csv('EOCC_Data_Robot_Input.csv')

In [32]:
#pca
pca1 = PCA(n_components=1)
pc_d_l = principalComponents = pca1.fit_transform(data[['CAR_AGE IN DAYS','LIFE TO DATE MILEAGE']])
data['PCA_d_l']=pc_d_l
#data = data.drop(['Mileage_since_last _failure','weeks_in_service'],axis = 1)

In [33]:
#pca
pca1 = PCA(n_components=1)
pc_w_m = principalComponents = pca1.fit_transform(data[['Mileage since Last Replacement','COMPONENT_AGE IN DAYS']])
data['PCA_w_m']=pc_w_m
#data = data.drop(['Mileage_since_last _failure','weeks_in_service'],axis = 1)

In [34]:
data.head()

,CAR NUMBER,LOCATION,Last Replacement Date,CURRENT_CONDITION_CODE,CUT_OFF_DATE,SHOP_INITIAL,Pocket_Type,Pocket_Number,Mileage since Last Replacement,CAR_CLASS,...,RR_ST,RR_UP,AVG_Loaded_Trip_Duration_Days,AVG_Loaded_Distance_Miles,AVG_Empty_Trip_Duration_Days,AVG_Empty_Distance_Miles,COMPONENT_AGE IN DAYS,CAR_AGE IN DAYS,PCA_d_l,PCA_w_m
0,83500,A,3/23/2018,C_1,3/6/2019,UP,0,NaN,21436,GSH64,...,0,12,20.329629,1060.389629,14.625925,581.204074,348,11934,222167.543748,-338011.264039
1,83500,B,6/6/2018,C_1,3/6/2019,UP,0,NaN,15750,GSH64,...,0,12,20.329629,1060.389629,14.625925,581.204074,273,11934,222167.543748,-343697.757931
2,83503,A,12/19/2014,C_3,3/6/2019,UP,0,NaN,69043,GSH64,...,0,19,30.188571,1083.472571,16.408571,802.380571,1538,11929,220313.572853,-290392.999194
3,83503,B,10/20/2016,C_3,3/6/2019,UP,0,NaN,30207,GSH64,...,0,19,30.188571,1083.472571,16.408571,802.380571,867,11929,220313.572853,-329234.386184
4,83505,A,6/22/2017,C_1,3/6/2019,UP,0,NaN,22552,GSH64,...,0,28,17.542857,1087.278367,17.858333,861.054375,622,11929,269908.601790,-336891.878117


In [35]:
data['CURRENT_CONDITION_CODE'] = pd.factorize(data.CURRENT_CONDITION_CODE)[0]
data['SHOP_INITIAL'] = pd.factorize(data.SHOP_INITIAL)[0]
data['Pocket_Type'] = pd.factorize(data.Pocket_Type)[0]
data['Pocket_Number'] = pd.factorize(data.Pocket_Number)[0]
data['CAR_CLASS'] = pd.factorize(data.CAR_CLASS)[0]
data['Equipment_Feature_Desc'] = pd.factorize(data.Equipment_Feature_Desc)[0]
data['BUILDER_NAME'] = pd.factorize(data.BUILDER_NAME)[0]

In [36]:
data.head()

,CAR NUMBER,LOCATION,Last Replacement Date,CURRENT_CONDITION_CODE,CUT_OFF_DATE,SHOP_INITIAL,Pocket_Type,Pocket_Number,Mileage since Last Replacement,CAR_CLASS,...,RR_ST,RR_UP,AVG_Loaded_Trip_Duration_Days,AVG_Loaded_Distance_Miles,AVG_Empty_Trip_Duration_Days,AVG_Empty_Distance_Miles,COMPONENT_AGE IN DAYS,CAR_AGE IN DAYS,PCA_d_l,PCA_w_m
0,83500,A,3/23/2018,0,3/6/2019,0,0,-1,21436,0,...,0,12,20.329629,1060.389629,14.625925,581.204074,348,11934,222167.543748,-338011.264039
1,83500,B,6/6/2018,0,3/6/2019,0,0,-1,15750,0,...,0,12,20.329629,1060.389629,14.625925,581.204074,273,11934,222167.543748,-343697.757931
2,83503,A,12/19/2014,1,3/6/2019,1,0,-1,69043,0,...,0,19,30.188571,1083.472571,16.408571,802.380571,1538,11929,220313.572853,-290392.999194
3,83503,B,10/20/2016,1,3/6/2019,0,0,-1,30207,0,...,0,19,30.188571,1083.472571,16.408571,802.380571,867,11929,220313.572853,-329234.386184
4,83505,A,6/22/2017,0,3/6/2019,0,0,-1,22552,0,...,0,28,17.542857,1087.278367,17.858333,861.054375,622,11929,269908.601790,-336891.878117


In [16]:
best_vals_reg=[]
best_vals_reg_pca = []
best_vals_reg_oversampling = []
best_vals_reg_oversampling_pca = []
for j in range(10):
    data_unique_car_number = pd.DataFrame(data['CAR NUMBER'].unique())
    data_unique_car_number.columns=['CAR NUMBER']
    
    data_random=pd.DataFrame(np.random.rand(len(data['CAR NUMBER'].unique())))
    data_random.columns=['RandomNum']
    
    data_unique_car_number['RandomNum']=pd.DataFrame(np.random.rand(len(data['CAR NUMBER'].unique())))
    
    data_merged =pd.merge(data,data_unique_car_number,on='CAR NUMBER',how='left')

    data_new = data_merged.drop(['CAR NUMBER','LOCATION', 'Last Replacement Date','CUT_OFF_DATE','BUILT DATE'],axis =1)
    
    #train and test  
    data_train_temp = data_new[data_new['RandomNum'] <= 0.75]
    data_train = data_train_temp.drop(['RandomNum'],axis =1)
    
    data_test_temp = data_new[data_new['RandomNum'] > 0.75]
    data_test = data_test_temp.drop(['RandomNum'],axis =1)
    
    ada = ADASYN(random_state=42)
    
    y_train = data_train['TARGET']
    x_train_pca = data_train.drop(['TARGET'],axis =1)
    x_train = data_train.drop(['TARGET','PCA_d_l','PCA_w_m'],axis =1)
    x_train_oversampling, y_train_oversampling = ada.fit_resample(x_train, y_train)
    x_train_oversampling_pca, y_train_oversampling_pca = ada.fit_resample(x_train_pca, y_train)

    y_test = data_test['TARGET']
    x_test = data_test.drop(['TARGET','PCA_d_l','PCA_w_m'],axis =1)
    x_test_pca = data_test.drop(['TARGET'],axis =1)
    
    
    clf_pca = RandomForestClassifier(random_state=1234,
                                    min_samples_split=2,
                                     max_leaf_nodes=3000,
                                     max_features=0.2,
                                     bootstrap = False, 
                                     n_estimators= 500,
                                     criterion='gini',
                                     min_samples_leaf=20).fit(x_train_pca,y_train)
    
    clf = RandomForestClassifier(random_state=1234,
                                    min_samples_split=2,
                                     max_leaf_nodes=3000,
                                     max_features=0.2,
                                     bootstrap = False, 
                                     n_estimators= 500,
                                     criterion='gini',
                                     min_samples_leaf=20).fit(x_train,y_train)
    
    clf_oversampling = RandomForestClassifier(random_state=1234,
                                    min_samples_split=2,
                                     max_leaf_nodes=3000,
                                     max_features=0.2,
                                     bootstrap = False, 
                                     n_estimators= 500,
                                     criterion='gini',
                                     min_samples_leaf=20).fit(x_train_oversampling,y_train_oversampling)
    
    clf_oversampling_pca = RandomForestClassifier(random_state=1234,
                                    min_samples_split=2,
                                     max_leaf_nodes=3000,
                                     max_features=0.2,
                                     bootstrap = False, 
                                     n_estimators= 500,
                                     criterion='gini',
                                     min_samples_leaf=20).fit(x_train_oversampling_pca,y_train_oversampling_pca)
    
    vals=[]
    for i in np.linspace(0.01,0.2,100):
        predicted_probas = clf.predict_proba(x_test)
        y_predict_test = (predicted_probas [:,1] >= i).astype('int')
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict_test)
        vals.append(metrics.auc(fpr, tpr))
    print('{0}-{1}'.format(max(vals),np.linspace(0.01,0.2,100)[vals.index(max(vals))]))
    best_vals_reg.append(max(vals)) 
    
    
    vals_pca=[]
    for i in np.linspace(0.01,0.2,100):
        predicted_probas = clf_pca.predict_proba(x_test_pca)
        y_predict_test = (predicted_probas [:,1] >= i).astype('int')
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict_test)
        vals_pca.append(metrics.auc(fpr, tpr))
    print('{0}-{1}'.format(max(vals_pca),np.linspace(0.01,0.2,100)[vals_pca.index(max(vals_pca))]))
    best_vals_reg_pca.append(max(vals_pca)) 
    
    
    vals_over=[]
    for i in np.linspace(0.01,0.2,100):
        predicted_probas = clf_oversampling.predict_proba(x_test)
        y_predict_test = (predicted_probas [:,1] >= i).astype('int')
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict_test)
        vals_over.append(metrics.auc(fpr, tpr))
    print('{0}-{1}'.format(max(vals_over),np.linspace(0.01,0.2,100)[vals_over.index(max(vals_over))]))
    best_vals_reg_oversampling.append(max(vals_over)) 
    
    vals_over_pca=[]
    for i in np.linspace(0.01,0.2,100):
        predicted_probas = clf_oversampling_pca.predict_proba(x_test_pca)
        y_predict_test = (predicted_probas [:,1] >= i).astype('int')
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict_test)
        vals_over_pca.append(metrics.auc(fpr, tpr))
    print('{0}-{1}'.format(max(vals_over_pca),np.linspace(0.01,0.2,100)[vals_over_pca.index(max(vals_over_pca))]))
    best_vals_reg_oversampling_pca.append(max(vals_over_pca)) 
    
    print('-------------------------------------------------------------')



0.664265650458773-0.03111111111111111
0.6715367132572039-0.027272727272727275
0.6383318952720716-0.1366666666666667
0.6322195557135382-0.1002020202020202
-------------------------------------------------------------
0.6666167570921884-0.04262626262626263
0.6761481574258046-0.025353535353535354
0.6175632361899449-0.19616161616161618
0.6121069621009844-0.19616161616161618
-------------------------------------------------------------
0.6579998588907149-0.04454545454545455
0.6543341301857662-0.046464646464646465
0.6458343709016053-0.1501010101010101
0.6408830951076581-0.15202020202020203
-------------------------------------------------------------
0.6631393629888456-0.027272727272727275
0.6575782824889128-0.027272727272727275
0.6231662410966268-0.18272727272727274
0.6215266765219729-0.15202020202020203
-------------------------------------------------------------
0.6542881559143898-0.03686868686868687
0.6538634218514834-0.03878787878787879
0.6260197443942757-0.12323232323232322
0.64205887

In [17]:
print(np.mean(best_vals_reg))
print(np.mean(best_vals_reg_pca))
print(np.mean(best_vals_reg_oversampling))
print(np.mean(best_vals_reg_oversampling_pca))

0.6667491106945492
0.6666410388335815
0.6376758357897802
0.6369250607526458


In [38]:
best_vals_reg=[]
for j in range(10):
    data_unique_car_number = pd.DataFrame(data['CAR NUMBER'].unique())
    data_unique_car_number.columns=['CAR NUMBER']
    
    data_random=pd.DataFrame(np.random.rand(len(data['CAR NUMBER'].unique())))
    data_random.columns=['RandomNum']
    
    data_unique_car_number['RandomNum']=pd.DataFrame(np.random.rand(len(data['CAR NUMBER'].unique())))
    
    data_merged =pd.merge(data,data_unique_car_number,on='CAR NUMBER',how='left')

    data_new = data_merged.drop(['CAR NUMBER','LOCATION', 'Last Replacement Date','CUT_OFF_DATE','BUILT DATE', 
                                 'CAR_AGE IN DAYS','LIFE TO DATE MILEAGE',
                                 'Mileage since Last Replacement','COMPONENT_AGE IN DAYS'],axis =1)
    
    #train and test  
    data_train_temp = data_new[data_new['RandomNum'] <= 0.75]
    data_train = data_train_temp.drop(['RandomNum'],axis =1)
    
    data_test_temp = data_new[data_new['RandomNum'] > 0.75]
    data_test = data_test_temp.drop(['RandomNum'],axis =1)
    
        #train and test  
    data_train_temp = data_new[data_new['RandomNum'] <= 0.75]
    data_train = data_train_temp.drop(['RandomNum'],axis =1)
    
    data_test_temp = data_new[data_new['RandomNum'] > 0.75]
    data_test = data_test_temp.drop(['RandomNum'],axis =1)
    
    
    y_train = data_train['TARGET']
    x_train = data_train.drop(['TARGET'],axis =1)

    
    y_test = data_test['TARGET']
    x_test = data_test.drop(['TARGET'],axis =1)
    
    
    
    clf = RandomForestClassifier(random_state=1234,
                                    min_samples_split=2,
                                     max_leaf_nodes=3000,
                                     max_features=0.2,
                                     bootstrap = False, 
                                     n_estimators= 500,
                                     criterion='gini',
                                     min_samples_leaf=20).fit(x_train,y_train)
    
    
    vals=[]
    for i in np.linspace(0.01,0.2,50):
        predicted_probas = clf.predict_proba(x_test)
        y_predict_test = (predicted_probas [:,1] >= i).astype('int')
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict_test)
        vals.append(metrics.auc(fpr, tpr))
    print('{0}-{1}'.format(max(vals),np.linspace(0.01,0.2,50)[vals.index(max(vals))]))
    best_vals_reg.append(max(vals)) 

0.61604870956016-0.041020408163265305
0.6247829313854028-0.037142857142857144
0.6220311505890759-0.03326530612244898
0.6017676040483159-0.03326530612244898
0.6276451347674329-0.03326530612244898
0.63682800422938-0.037142857142857144
0.5982547356258932-0.03326530612244898
0.6128529973812725-0.029387755102040815
0.6051246070947462-0.052653061224489796
0.643086262282691-0.037142857142857144


In [39]:
print(np.mean(best_vals_reg))

0.6188422136964371
